In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
import numpy as np
from pprint import pprint
from tqdm.auto import tqdm
from tqdm import tqdm_notebook
from scipy.sparse import hstack
from collections import defaultdict, Counter
from ds_tools.ds_tools import CategoricalTransformer
tqdm.pandas()

In [2]:
df_description = pd.read_csv('./data/data_definition.txt', sep='\t')

df_train_genba = pd.read_csv('./data/train_genba.tsv', sep='\t')
df_train_goto = pd.read_csv('./data/train_goto.tsv', sep='\t')

df_train = df_train_goto.merge(df_train_genba, on='pj_no', how='left')
df_train.drop('id', axis=1, inplace=True)

df_test_genba = pd.read_csv('./data/test_genba.tsv', sep='\t')
df_test_goto = pd.read_csv('./data/test_goto.tsv', sep='\t')

df_test = df_test_goto.merge(df_test_genba, on='pj_no', how='left')
test_surface = df_test['tc_mseki']
df_test.drop('id', axis=1, inplace=True)

## Preprocessing

In [3]:
def fill_city_name(name):
    if '市' not in name and '郡' not in name:
        name = '市' + name
    return name

def split_address(df):
    df['jukyo'] = df['jukyo'].str.slice(start=3).str.replace(r'[ヶｹ]', 'ケ')
    df['jukyo'] = df['jukyo'].apply(fill_city_name)
    city_split = df['jukyo'].str.split(r'[市郡]', n=1, expand=True)
    df['city'] = city_split[0]
    street_split = city_split[1].str.split(r'[町区]', n=1, expand=True)
    df['street'] = street_split[0]
    df['address_detail'] = street_split[1].str.strip().replace('', None)
    return df

df_train = split_address(df_train)
df_test = split_address(df_test)

df_train.drop(['kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)
df_test.drop(['kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)

In [4]:
def combine(row, combine_list, tup):
    l = set()
    for col in tup:
        if pd.notnull(row[col]):
            l.add(row[col])
    combine_list.append(','.join(l))


combine_cols = [('yoto', 100), ('road_hk', 100)]
for i, tup in enumerate([['yoto1', 'yoto2'], ['road1_hk', 'road2_hk', 'road3_hk', 'road4_hk']]):
    combine_train = []
    combine_test = []
    
    combine_col_name = combine_cols[i][0]
    _ = df_train.apply(lambda row: combine(row, combine_train, tup), axis=1)
    _ = df_test.apply(lambda row: combine(row, combine_test, tup), axis=1)

    count_vectorizer = CountVectorizer(min_df=combine_cols[i][1])
    combine_train_matrix = count_vectorizer.fit_transform(combine_train).todense()
    combine_test_matrix = count_vectorizer.transform(combine_test).todense()
    for i in range(combine_train_matrix.shape[1]):
        df_train['%s_%d' % (combine_col_name, i)] = combine_train_matrix[:, i]
        df_test['%s_%d' % (combine_col_name, i)] = combine_test_matrix[:, i]
    for col in tup:
        df_train.drop(col, axis=1, inplace=True)
        df_test.drop(col, axis=1, inplace=True)

In [5]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
price_stats = []
for train_idx, valid_idx in splitter.split(df_train):
    price_stats_by_city = defaultdict(dict)
    for city, group in df_train.iloc[train_idx].groupby('city'):
        price_list = group['keiyaku_pr']/group['tc_mseki']
        price_stats_by_city[city]['price_by_city_mean'] = price_list.mean()
        price_stats_by_city[city]['price_by_city_median'] = price_list.median()
        price_stats_by_city[city]['price_by_city_min'] = price_list.min()
        price_stats_by_city[city]['price_by_city_max'] = price_list.max()
        price_stats_by_city[city]['price_by_city_std'] = price_list.std()
        price_stats_by_city[city]['price_by_city_count'] = len(price_list)
    for i, city in enumerate(df_train.iloc[valid_idx]['city']):
        price_stats.append((valid_idx[i], price_stats_by_city[city]))

price_stats_test = [] 
price_stats_by_city = defaultdict(dict)
for city, group in df_train.groupby('city'):
    price_list = group['keiyaku_pr']/group['tc_mseki']
    price_stats_by_city[city]['price_by_city_mean'] = price_list.mean()
    price_stats_by_city[city]['price_by_city_median'] = price_list.median()
    price_stats_by_city[city]['price_by_city_min'] = price_list.min()
    price_stats_by_city[city]['price_by_city_max'] = price_list.max()
    price_stats_by_city[city]['price_by_city_std'] = price_list.std()
    price_stats_by_city[city]['price_by_city_count'] = len(price_list)
for city in df_test['city']:
    price_stats_test.append(price_stats_by_city[city])
    
df_price_stats = pd.DataFrame([x[1] for x in sorted(price_stats, key=lambda x: x[0])])
df_price_stats_test = pd.DataFrame(price_stats_test)

df_train = pd.concat([df_train, df_price_stats], axis=1)
df_test = pd.concat([df_test, df_price_stats_test], axis=1)

for col in ['mseki_rd_hb', 'road3_fi', 'rosenka_hb', 'kempei2', 'road2_mg', 'kaoku_hb', 'bus_hon']:
    df_train[col].replace(0.0, np.nan, inplace=True)
    df_test[col].replace(0.0, np.nan, inplace=True)

## Train

In [6]:
continue_features = list(df_description[(df_description['データ型'] == '数値') & (df_description['項目名'] != 'pj_no')]['項目名'])
continue_features += ['price_by_city_mean', 'price_by_city_median', 'price_by_city_min', 'price_by_city_max', 
                      'price_by_city_std', 'price_by_city_count']
objective = 'keiyaku_pr'
categorical_features = list(df_train)

for f in continue_features+[objective]:
    if f in categorical_features:
        categorical_features.remove(f)
        
for col in categorical_features:
    if col not in ['pj_no']:
        ct = CategoricalTransformer(min_freq=3)
        df_train[col] = ct.fit_transform(df_train[col])
        df_test[col] = ct.transform(df_test[col])
        
for col in continue_features:
    if col != 'keiyaku_pr':
        scaler = MinMaxScaler()
        df_train[col] = scaler.fit_transform(df_train[col].values.reshape(-1, 1))
        df_test[col] = scaler.transform(df_test[col].values.reshape(-1, 1))
    
df_test['keiyaku_pr'] = 0

## LGB

In [7]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    X_train, y_train = train.drop('keiyaku_pr', axis=1), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = valid.drop('keiyaku_pr', axis=1), np.log(valid['keiyaku_pr']+1)
    regressor = lgb.LGBMRegressor(n_estimators=20000, learning_rate=0.01, silent=False, random_state=28,
                                 bagging_fraction=0.9, bagging_freq=1, feature_fraction=0.8, lambda_l1=0.01)
    regressor.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=500,
                  categorical_feature=categorical_features)
    prediction_list.append(regressor.predict(df_test[df_train.drop(objective, axis=1).columns]))
    best_scores.append(regressor.best_score_['valid_0']['l2'])

print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission.tsv', sep='\t', header=None, index=False)

# 0.01210079

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['address_detail', 'bas_toho1', 'bas_toho2', 'bastei_nm1', 'bastei_nm2', 'bokachiiki', 'bus_yohi', 'chikukeikaku', 'city', 'eki_nm1', 'eki_nm2', 'fi3m_yohi', 'fi4m_yohi', 'gas', 'gesui', 'hiatari', 'hokakisei1', 'hokakisei2', 'hokakisei3', 'hokakisei4', 'hokakyoka', 'hw_status', 'jigata', 'josui', 'jukyo', 'kaihatsukyoka', 'kborjs', 'keikakuroad', 'kinshijiko', 'kobetsu1', 'kobetsu2', 'kobetsu3', 'kobetsu4', 'kodochiku', 'levelplan', 'pj_no', 'road1_sb', 'road2_sb', 'road3_sb', 'road4_sb', 'road_hk_0', 'road_hk_1', 'road_hk_2', 'road_hk_3', 'road_st', 'rosen_nm1', 'rosen_nm2', 'rs_e_kdate2', 'rs_e_kdate3', 'rs_e_m_ari', 'rs_e_m_nashi', 'rs_e_parking', 'rs_e_tahata', 'rs_e_zoki', 'rs_n_kdate2', 'rs_n_kdate3', 'rs_n_m_ari', 'rs_n_m_nashi', 'rs_n_parking', 'rs_n_tahata', 'rs_n_zoki', 'rs_s_kdate2', 'rs_s_kdate3', 'rs_s_m_ari', 'rs_s_m_nashi',

[1]	valid_0's l2: 0.0676443
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's l2: 0.0669453
[3]	valid_0's l2: 0.0661952
[4]	valid_0's l2: 0.0654631
[5]	valid_0's l2: 0.064737
[6]	valid_0's l2: 0.0640547
[7]	valid_0's l2: 0.0633642
[8]	valid_0's l2: 0.0626802
[9]	valid_0's l2: 0.0620088
[10]	valid_0's l2: 0.0613955
[11]	valid_0's l2: 0.0607645
[12]	valid_0's l2: 0.0601487
[13]	valid_0's l2: 0.0595241
[14]	valid_0's l2: 0.058963
[15]	valid_0's l2: 0.0583617
[16]	valid_0's l2: 0.0577917
[17]	valid_0's l2: 0.057212
[18]	valid_0's l2: 0.0566641
[19]	valid_0's l2: 0.0561033
[20]	valid_0's l2: 0.0555909
[21]	valid_0's l2: 0.0550535
[22]	valid_0's l2: 0.0545331
[23]	valid_0's l2: 0.0540307
[24]	valid_0's l2: 0.0535331
[25]	valid_0's l2: 0.0530187
[26]	valid_0's l2: 0.052529
[27]	valid_0's l2: 0.0520352
[28]	valid_0's l2: 0.0515469
[29]	valid_0's l2: 0.051077
[30]	valid_0's l2: 0.0506165
[31]	valid_0's l2: 0.0501823
[32]	valid_0's l2: 0.0497511
[33]	valid_0's l2: 0.04

[321]	valid_0's l2: 0.0202577
[322]	valid_0's l2: 0.0202381
[323]	valid_0's l2: 0.0202203
[324]	valid_0's l2: 0.020208
[325]	valid_0's l2: 0.0201939
[326]	valid_0's l2: 0.0201805
[327]	valid_0's l2: 0.020163
[328]	valid_0's l2: 0.0201482
[329]	valid_0's l2: 0.0201368
[330]	valid_0's l2: 0.0201264
[331]	valid_0's l2: 0.020111
[332]	valid_0's l2: 0.020095
[333]	valid_0's l2: 0.0200755
[334]	valid_0's l2: 0.0200589
[335]	valid_0's l2: 0.0200465
[336]	valid_0's l2: 0.0200324
[337]	valid_0's l2: 0.0200146
[338]	valid_0's l2: 0.02
[339]	valid_0's l2: 0.0199858
[340]	valid_0's l2: 0.0199747
[341]	valid_0's l2: 0.0199642
[342]	valid_0's l2: 0.0199516
[343]	valid_0's l2: 0.0199404
[344]	valid_0's l2: 0.0199252
[345]	valid_0's l2: 0.019914
[346]	valid_0's l2: 0.0199024
[347]	valid_0's l2: 0.0198891
[348]	valid_0's l2: 0.0198779
[349]	valid_0's l2: 0.019865
[350]	valid_0's l2: 0.0198498
[351]	valid_0's l2: 0.0198346
[352]	valid_0's l2: 0.0198215
[353]	valid_0's l2: 0.0198074
[354]	valid_0's l2: 0

[603]	valid_0's l2: 0.0180542
[604]	valid_0's l2: 0.0180544
[605]	valid_0's l2: 0.0180525
[606]	valid_0's l2: 0.0180485
[607]	valid_0's l2: 0.0180423
[608]	valid_0's l2: 0.0180372
[609]	valid_0's l2: 0.0180312
[610]	valid_0's l2: 0.0180231
[611]	valid_0's l2: 0.0180203
[612]	valid_0's l2: 0.0180174
[613]	valid_0's l2: 0.0180093
[614]	valid_0's l2: 0.0180089
[615]	valid_0's l2: 0.0180042
[616]	valid_0's l2: 0.0180001
[617]	valid_0's l2: 0.0179976
[618]	valid_0's l2: 0.0179937
[619]	valid_0's l2: 0.0179879
[620]	valid_0's l2: 0.0179857
[621]	valid_0's l2: 0.0179838
[622]	valid_0's l2: 0.0179802
[623]	valid_0's l2: 0.0179762
[624]	valid_0's l2: 0.0179705
[625]	valid_0's l2: 0.017968
[626]	valid_0's l2: 0.017965
[627]	valid_0's l2: 0.0179628
[628]	valid_0's l2: 0.0179606
[629]	valid_0's l2: 0.0179581
[630]	valid_0's l2: 0.0179547
[631]	valid_0's l2: 0.0179519
[632]	valid_0's l2: 0.0179478
[633]	valid_0's l2: 0.0179448
[634]	valid_0's l2: 0.0179428
[635]	valid_0's l2: 0.0179408
[636]	valid_

[893]	valid_0's l2: 0.0174212
[894]	valid_0's l2: 0.0174189
[895]	valid_0's l2: 0.0174185
[896]	valid_0's l2: 0.0174184
[897]	valid_0's l2: 0.0174179
[898]	valid_0's l2: 0.0174167
[899]	valid_0's l2: 0.0174181
[900]	valid_0's l2: 0.0174158
[901]	valid_0's l2: 0.0174151
[902]	valid_0's l2: 0.0174148
[903]	valid_0's l2: 0.0174143
[904]	valid_0's l2: 0.0174128
[905]	valid_0's l2: 0.0174137
[906]	valid_0's l2: 0.0174104
[907]	valid_0's l2: 0.0174095
[908]	valid_0's l2: 0.0174104
[909]	valid_0's l2: 0.0174085
[910]	valid_0's l2: 0.0174066
[911]	valid_0's l2: 0.017406
[912]	valid_0's l2: 0.0174064
[913]	valid_0's l2: 0.0174073
[914]	valid_0's l2: 0.0174069
[915]	valid_0's l2: 0.0174051
[916]	valid_0's l2: 0.0174041
[917]	valid_0's l2: 0.0174015
[918]	valid_0's l2: 0.0174004
[919]	valid_0's l2: 0.0173997
[920]	valid_0's l2: 0.0173993
[921]	valid_0's l2: 0.0173999
[922]	valid_0's l2: 0.0173991
[923]	valid_0's l2: 0.0174003
[924]	valid_0's l2: 0.0173996
[925]	valid_0's l2: 0.0173987
[926]	valid

...

[2806]	valid_0's l2: 0.0114889
[2807]	valid_0's l2: 0.0114894
[2808]	valid_0's l2: 0.0114888
[2809]	valid_0's l2: 0.0114889
[2810]	valid_0's l2: 0.0114882
[2811]	valid_0's l2: 0.0114876
[2812]	valid_0's l2: 0.0114875
[2813]	valid_0's l2: 0.0114865
[2814]	valid_0's l2: 0.0114863
[2815]	valid_0's l2: 0.011486
[2816]	valid_0's l2: 0.0114864
[2817]	valid_0's l2: 0.0114871
[2818]	valid_0's l2: 0.011487
[2819]	valid_0's l2: 0.0114866
[2820]	valid_0's l2: 0.011487
[2821]	valid_0's l2: 0.0114861
[2822]	valid_0's l2: 0.0114865
[2823]	valid_0's l2: 0.0114867
[2824]	valid_0's l2: 0.0114866
[2825]	valid_0's l2: 0.0114866
[2826]	valid_0's l2: 0.0114867
[2827]	valid_0's l2: 0.0114868
[2828]	valid_0's l2: 0.0114864
[2829]	valid_0's l2: 0.0114864
[2830]	valid_0's l2: 0.0114861
[2831]	valid_0's l2: 0.0114865
[2832]	valid_0's l2: 0.0114871
[2833]	valid_0's l2: 0.0114868
[2834]	valid_0's l2: 0.0114861
[2835]	valid_0's l2: 0.0114867
[2836]	valid_0's l2: 0.0114877
[2837]	valid_0's l2: 0.0114884
[2838]	vali

[3078]	valid_0's l2: 0.0114763
[3079]	valid_0's l2: 0.0114766
[3080]	valid_0's l2: 0.0114768
[3081]	valid_0's l2: 0.0114774
[3082]	valid_0's l2: 0.0114773
[3083]	valid_0's l2: 0.0114761
[3084]	valid_0's l2: 0.011477
[3085]	valid_0's l2: 0.011476
[3086]	valid_0's l2: 0.0114769
[3087]	valid_0's l2: 0.0114774
[3088]	valid_0's l2: 0.0114774
[3089]	valid_0's l2: 0.0114773
[3090]	valid_0's l2: 0.0114764
[3091]	valid_0's l2: 0.0114769
[3092]	valid_0's l2: 0.0114765
[3093]	valid_0's l2: 0.0114767
[3094]	valid_0's l2: 0.0114772
[3095]	valid_0's l2: 0.0114768
[3096]	valid_0's l2: 0.0114775
[3097]	valid_0's l2: 0.011477
[3098]	valid_0's l2: 0.011477
[3099]	valid_0's l2: 0.0114773
[3100]	valid_0's l2: 0.0114774
[3101]	valid_0's l2: 0.0114775
[3102]	valid_0's l2: 0.0114782
[3103]	valid_0's l2: 0.0114782
[3104]	valid_0's l2: 0.0114782
[3105]	valid_0's l2: 0.0114784
[3106]	valid_0's l2: 0.0114782
[3107]	valid_0's l2: 0.0114778
[3108]	valid_0's l2: 0.0114775
[3109]	valid_0's l2: 0.011478
[3110]	valid_

[3353]	valid_0's l2: 0.0114647
[3354]	valid_0's l2: 0.0114647
[3355]	valid_0's l2: 0.0114645
[3356]	valid_0's l2: 0.0114645
[3357]	valid_0's l2: 0.0114649
[3358]	valid_0's l2: 0.0114648
[3359]	valid_0's l2: 0.0114644
[3360]	valid_0's l2: 0.0114641
[3361]	valid_0's l2: 0.0114642
[3362]	valid_0's l2: 0.0114644
[3363]	valid_0's l2: 0.0114642
[3364]	valid_0's l2: 0.0114638
[3365]	valid_0's l2: 0.0114633
[3366]	valid_0's l2: 0.0114623
[3367]	valid_0's l2: 0.0114617
[3368]	valid_0's l2: 0.0114606
[3369]	valid_0's l2: 0.0114611
[3370]	valid_0's l2: 0.0114612
[3371]	valid_0's l2: 0.0114617
[3372]	valid_0's l2: 0.0114618
[3373]	valid_0's l2: 0.0114614
[3374]	valid_0's l2: 0.011462
[3375]	valid_0's l2: 0.0114603
[3376]	valid_0's l2: 0.0114601
[3377]	valid_0's l2: 0.0114601
[3378]	valid_0's l2: 0.0114601
[3379]	valid_0's l2: 0.01146
[3380]	valid_0's l2: 0.0114603
[3381]	valid_0's l2: 0.0114601
[3382]	valid_0's l2: 0.01146
[3383]	valid_0's l2: 0.0114594
[3384]	valid_0's l2: 0.01146
[3385]	valid_0'

[3685]	valid_0's l2: 0.0114409
[3686]	valid_0's l2: 0.0114386
[3687]	valid_0's l2: 0.0114387
[3688]	valid_0's l2: 0.0114391
[3689]	valid_0's l2: 0.0114393
[3690]	valid_0's l2: 0.0114394
[3691]	valid_0's l2: 0.0114394
[3692]	valid_0's l2: 0.0114392
[3693]	valid_0's l2: 0.0114393
[3694]	valid_0's l2: 0.0114391
[3695]	valid_0's l2: 0.0114389
[3696]	valid_0's l2: 0.011439
[3697]	valid_0's l2: 0.0114389
[3698]	valid_0's l2: 0.0114391
[3699]	valid_0's l2: 0.0114392
[3700]	valid_0's l2: 0.0114393
[3701]	valid_0's l2: 0.0114394
[3702]	valid_0's l2: 0.0114404
[3703]	valid_0's l2: 0.0114394
[3704]	valid_0's l2: 0.0114395
[3705]	valid_0's l2: 0.0114395
[3706]	valid_0's l2: 0.0114398
[3707]	valid_0's l2: 0.0114399
[3708]	valid_0's l2: 0.0114404
[3709]	valid_0's l2: 0.0114406
[3710]	valid_0's l2: 0.0114407
[3711]	valid_0's l2: 0.0114407
[3712]	valid_0's l2: 0.0114409
[3713]	valid_0's l2: 0.0114409
[3714]	valid_0's l2: 0.011441
[3715]	valid_0's l2: 0.0114405
[3716]	valid_0's l2: 0.0114407
[3717]	val

[3952]	valid_0's l2: 0.0114456
[3953]	valid_0's l2: 0.0114456
[3954]	valid_0's l2: 0.0114459
[3955]	valid_0's l2: 0.0114464
[3956]	valid_0's l2: 0.0114465
[3957]	valid_0's l2: 0.0114465
[3958]	valid_0's l2: 0.0114464
[3959]	valid_0's l2: 0.0114463
[3960]	valid_0's l2: 0.0114457
[3961]	valid_0's l2: 0.0114459
[3962]	valid_0's l2: 0.011446
[3963]	valid_0's l2: 0.0114451
[3964]	valid_0's l2: 0.0114447
[3965]	valid_0's l2: 0.0114443
[3966]	valid_0's l2: 0.0114441
[3967]	valid_0's l2: 0.0114443
[3968]	valid_0's l2: 0.0114442
[3969]	valid_0's l2: 0.0114443
[3970]	valid_0's l2: 0.0114443
[3971]	valid_0's l2: 0.0114442
[3972]	valid_0's l2: 0.0114443
[3973]	valid_0's l2: 0.0114438
[3974]	valid_0's l2: 0.0114441
[3975]	valid_0's l2: 0.0114444
[3976]	valid_0's l2: 0.0114445
[3977]	valid_0's l2: 0.0114446
[3978]	valid_0's l2: 0.0114447
[3979]	valid_0's l2: 0.0114445
[3980]	valid_0's l2: 0.0114445
[3981]	valid_0's l2: 0.0114448
[3982]	valid_0's l2: 0.0114449
[3983]	valid_0's l2: 0.011445
[3984]	val

## Ridge

In [10]:
from sklearn import linear_model

continue_features.remove('keiyaku_pr')
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
df_train.fillna(0, inplace=True)
df_test.fillna(0, inplace=True)
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    test = df_test[df_train.drop(objective, axis=1).columns]
    encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
    train_cat_one_hot = encoder.fit_transform(train[categorical_features])
    valid_cat_one_hot = encoder.transform(valid[categorical_features])
    test_cat_one_hot = encoder.transform(test[categorical_features])
    X_train, y_train = hstack([train[continue_features].values, train_cat_one_hot]), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = hstack([valid[continue_features].values, valid_cat_one_hot]), np.log(valid['keiyaku_pr']+1)
    regressor = linear_model.Ridge(alpha=2, random_state=28)
    regressor.fit(X_train, y_train)
    pred_val = regressor.predict(X_valid)
    prediction_list.append(regressor.predict(hstack([test[continue_features].values, test_cat_one_hot])))
    best_scores.append(mean_squared_error(y_valid, pred_val))
    
print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission_ridge.tsv', sep='\t', header=None, index=False)

# 0.01408214

5-fold cv mean l2 0.01408214


## HuberRegressor

In [11]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    test = df_test[df_train.drop(objective, axis=1).columns]
    encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
    train_cat_one_hot = encoder.fit_transform(train[categorical_features])
    valid_cat_one_hot = encoder.transform(valid[categorical_features])
    test_cat_one_hot = encoder.transform(test[categorical_features])
    X_train, y_train = hstack([train[continue_features].values, train_cat_one_hot]), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = hstack([valid[continue_features].values, valid_cat_one_hot]), np.log(valid['keiyaku_pr']+1)
    regressor = linear_model.HuberRegressor(max_iter=500, epsilon=1.2, alpha=0.00001)
    regressor.fit(X_train, y_train)
    pred_val = regressor.predict(X_valid)
    prediction_list.append(regressor.predict(hstack([test[continue_features].values, test_cat_one_hot])))
    best_scores.append(mean_squared_error(y_valid, pred_val))
    
print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission_huber.tsv', sep='\t', header=None, index=False)

# 0.01366592

5-fold cv mean l2 0.01366592


## Linear svr

In [12]:
from sklearn import svm

splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    test = df_test[df_train.drop(objective, axis=1).columns]
    encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
    train_cat_one_hot = encoder.fit_transform(train[categorical_features])
    valid_cat_one_hot = encoder.transform(valid[categorical_features])
    test_cat_one_hot = encoder.transform(test[categorical_features])
    X_train, y_train = hstack([train[continue_features].values, train_cat_one_hot]), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = hstack([valid[continue_features].values, valid_cat_one_hot]), np.log(valid['keiyaku_pr']+1)
    regressor = svm.LinearSVR(C=0.1, epsilon=0.01, random_state=28)
    regressor.fit(X_train, y_train)
    pred_val = regressor.predict(X_valid)
    prediction_list.append(regressor.predict(hstack([test[continue_features].values, test_cat_one_hot])))
    best_scores.append(mean_squared_error(y_valid, pred_val))
    
print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission_linear_svr.tsv', sep='\t', header=None, index=False)

# 0.01330335

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


5-fold cv mean l2 0.01330335


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


## Merge

In [12]:
df_1 = pd.read_csv('./submission.tsv', sep='\t', names=['id', 'pred'])     #0.01210079
df_2 = pd.read_csv('./submission_ridge.tsv', sep='\t', names=['id', 'pred'])   #0.01408214
df_3 = pd.read_csv('./submission_huber.tsv', sep='\t', names=['id', 'pred'])  #0.01366592
df_4 = pd.read_csv('./submission_linear_svr.tsv', sep='\t', names=['id', 'pred'])  #0.01330335

df_1['pred'] = df_1['pred']*0.75 + df_2['pred']*0.025 + df_3['pred']*0.075 + df_4['pred']*0.15
df_1.to_csv('submission_merge.tsv', sep='\t', index=False, header=None) 